In [1]:
# %load imdb_data_to_graph.py
import numpy as np
import dgl
import pickle
import os
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import sys
import argparse
import copy
import itertools

In [2]:
# The following works for homogenous graphs, where possibly different edges are represented with eids.
dataset='wn18'

The following loads the graph for the kg datasets

In [ ]:
sys.path.append('../aux_files/kg/') 
from dataloader import get_dataset
from dataloader import EvalDataset, TrainDataset, NewBidirectionalOneShotIterator
datasetkg = get_dataset('../aux_files/kg/data', dataset, 'built_in',None)
train_edges=datasetkg.train
sin=train_edges[0]
oin=train_edges[2]
data_folder = '../data/kg/' +dataset + '/'
s=np.append(sin,oin).astype(int)
o=np.append(oin,sin).astype(int)
merged_list = list(tuple(zip(list(s),list(o))))
edges=list(set(merged_list))
np.savetxt(data_folder+"edge_list_complete.txt",np.array(edges,dtype=int),fmt='%i')

The following loads the graph for the rgcn datasets

In [5]:
sys.path.append('../aux_files/rgcn/') 
from dgl.contrib.data import load_data
data = load_data(dataset)
train_edges = data.train
sin = train_edges[:,0]
oin = train_edges[:,2]
data_folder = '../../data/kg/' + dataset + '/'
s=np.append(sin,oin).astype(int)
o=np.append(oin,sin).astype(int)
merged_list = list(tuple(zip(list(s),list(o))))
edgesr=list(set(merged_list))
np.savetxt(data_folder+"edge_list_complete.txt",np.array(edgesr,dtype=int),fmt='%i')

# entities: 40943
# relations: 18
# edges: 141442


In [ ]:
set(edges)==set(edgesr)

For the IMDB type of graph where a lot of relationships are among different entinties the motif representation
will be quite poor. Consider combining several relationtypes to get more interesting signals...

In [7]:
folder='../../../../PGD-orbit-master/'
etype='complete'

# the row index corresponds to the original id and the value corresponds to the internal id used by nasreen
vertex_mapping_2_nasreens_ids=np.loadtxt(folder+"vertex_mapping.txt") 
mapping_from_nasreens_ids={int(vertex_mapping_2_nasreens_ids[i]): i for i in range(len(vertex_mapping_2_nasreens_ids))}
motif_per_node=np.loadtxt(folder+dataset+'_'+etype+"-node.txt",skiprows=1,delimiter=',')

In [8]:
etype='complete'
motif_per_edge=np.loadtxt(folder+dataset+'_'+etype+"-edge.txt",skiprows=1,delimiter=',')


    

Some nodes are disconnected, for this the mapping from nasreen will be -1

In [9]:
print(len(np.unique(vertex_mapping_2_nasreens_ids)))
print(len(vertex_mapping_2_nasreens_ids))
condition = (vertex_mapping_2_nasreens_ids==-1)
print(len(vertex_mapping_2_nasreens_ids[condition]))

40943
40943
0


Nasreens code returns bidirectional graph by considering both directions of the edges of the original directed graphs.
Hence the returned graph has some edges that do not exist in the original graph. For now I just skip these.

The following creates a new edge, called motif that has the motif data.

In [10]:
(motif_per_edge).shape

(75451, 26)

In [20]:
sum(motif_per_edge)

array([1.52695820e+09, 7.23424081e+08, 2.54266000e+05, 3.04198200e+06,
       2.95139000e+06, 1.63674000e+05, 1.63674000e+05, 2.95139000e+06,
       1.51410000e+04, 1.01129030e+07, 3.00826100e+06, 6.56058200e+06,
       3.37828140e+08, 4.35342000e+05, 4.35342000e+05, 3.37828140e+08,
       2.57200000e+05, 2.88585000e+05, 1.01260000e+04, 2.98711000e+05,
       1.01260000e+04, 2.88585000e+05, 5.31280000e+04, 3.42800000e+03,
       1.41390000e+04, 3.48000000e+02])

In [21]:
# get all triangles
# Triangle id = 8
colum_id=8
etype='triangle'
dict_motif_edges={}
dict_motif_edata={}
threshold=1
for i in range(motif_per_edge.shape[0]):
    
    homo_id_dest=int(mapping_from_nasreens_ids[motif_per_edge[i,0]-1])
    homo_id_src=int(mapping_from_nasreens_ids[motif_per_edge[i,1]-1])
    #print('--'+str(homo_id))
    if motif_per_edge[i,colum_id]>=threshold:
        if etype not in dict_motif_edges:
            dict_motif_edges[etype]=[(homo_id_src,homo_id_dest)]
        else:
            dict_motif_edges[etype]+=[(homo_id_src,homo_id_dest)]

In [22]:
print(len(dict_motif_edges[etype]))
print(len(motif_per_edge))

11872
75451


In [23]:
npedges=np.array(dict_motif_edges[etype])
nodes=np.concatenate((npedges[:,0],npedges[:,1]))
len(np.unique(nodes))

8001

In [24]:
data_folder="../../data/kg/"+dataset+"/"
np.savetxt(data_folder+"tr"+str(threshold)+"_motif_edge_list.txt",dict_motif_edges[etype],fmt='%i')

In [ ]:
edata=[]
src_id=[]
dest_id=[]
#v,u,O1_v,O1_u,O2_v,O2_u,O3_v,O3_u,O4,O5_v,O5_u,O6,O7_v,O7_u,O8_v,O8_u,O9,O10_v,O10_u,O11,O12_v,O12_u,O13_v,O13_u,O14,O15
#edge_motifs_only=[8,11,16]
for i in range(motif_per_edge.shape[0]):
    homo_id_dest=int(mapping_from_nasreens_ids[motif_per_edge[i,0]-1])
    homo_id_src=int(mapping_from_nasreens_ids[motif_per_edge[i,1]-1])
    src_id+=[homo_id_src]
    dest_id+=[homo_id_dest]
    edata+=[torch.tensor(motif_per_edge[i,2:]).int()]
edata=torch.stack(edata)
src_id=torch.tensor(src_id).int()
dest_id=torch.tensor(dest_id).int()

In [ ]:
dest_id

In [ ]:
dict_motif_edata={'edata':edata,'src':src_id,'dest':dest_id}

In [ ]:
pickle.dump(dict_motif_edata, open(os.path.join(data_folder, "edge_motifs.pickle"), "wb"),
                protocol=4);